In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [2]:
parquet_file = "all_yellow_tripdata_filtered.parquet"
parquet_reader = pq.ParquetFile(parquet_file)

In [4]:
table = pq.read_table(parquet_file)
df = table.to_pandas()


In [5]:
num_rows = parquet_reader.metadata.num_rows
num_row_groups = parquet_reader.num_row_groups
df.shape[0]

118425410

In [6]:
print("Number of rows:", num_rows)
print("Number of cols:", df.shape[1])
print("Data types:") 
print(df.dtypes)

Number of rows: 118425410
Number of cols: 5
Data types:
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
PULocationID                      int64
DOLocationID                      int64
dtype: object


In [7]:
category_columns = ['PULocationID', 'DOLocationID']
for column in category_columns:
    df[column] = df[column].astype('category')

In [8]:
# Numeric
numeric_columns = df.select_dtypes(['int64', 'float64', 'datetime64']).columns

# Data Cleaning

Drop duplicates

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.shape

(117640646, 5)

Drop rows where pickup time before dropoff

In [20]:
df.drop(df[df['tpep_dropoff_datetime']<df['tpep_pickup_datetime']].index, inplace=True)

In [21]:
df.shape

(117586615, 5)

Drop rows where pickup time is before 2021

In [22]:
df.drop(df[df['tpep_pickup_datetime']<pd.to_datetime('2021-01-01')].index, inplace=True)

In [23]:
df.shape

(117585713, 5)

Drop rows where dropoff time is after 31st Mar 2024

In [24]:
df.drop(df[df['tpep_dropoff_datetime']>pd.to_datetime('2024-04-01')].index, inplace=True)

In [25]:
df.shape

(117585103, 5)

Drop rows where passenger count is missing

In [17]:
df[df['passenger_count'].isnull()]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
1271417,2021-01-01 00:06:47,2021-01-01 00:54:51,NaN,49,86
1271418,2021-01-01 00:43:03,2021-01-01 01:20:56,NaN,45,15
1271419,2021-01-01 00:35:18,2021-01-01 00:40:52,NaN,262,237
1271420,2021-01-01 00:38:00,2021-01-01 00:51:00,NaN,137,142
1271421,2021-01-01 00:13:47,2021-01-01 00:39:18,NaN,226,219
...,...,...,...,...,...
118425405,2024-03-31 23:16:45,2024-03-31 23:29:20,NaN,130,218
118425406,2024-03-31 23:29:28,2024-03-31 23:43:47,NaN,79,100
118425407,2024-03-31 23:15:00,2024-03-31 23:47:29,NaN,63,181
118425408,2024-03-31 23:27:53,2024-03-31 23:45:44,NaN,161,148


In [26]:
df.drop(df[df['passenger_count'].isnull()].index, inplace=True)

In [27]:
df.shape

(112728778, 5)

Drop rows where trip duration is > 5hrs

In [28]:
time_diff = pd.Timedelta(hours=5)
df.drop(df[df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'] > time_diff].index, inplace=True)

In [29]:
df.shape

(112587401, 5)

Drop rows where passenger count is 0

In [30]:
df.drop(df[df['passenger_count']==0].index, inplace=True)


In [31]:
df.shape

(110431491, 5)

Drop rows where passenger count is >6

In [32]:
df.drop(df[df['passenger_count']>6].index, inplace=True)

In [33]:
df.shape

(110430431, 5)

In [ ]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [34]:
df.to_csv("all_yellow_tripdata_cleaned.csv", index=False)

In [35]:
df.to_parquet("all_yellow_tripdata_cleaned.parquet", engine='pyarrow', index=False)

### Logical integrity tests
1. dropoff not before pickup
2. Passenger count not negative
3. Passenger count not greater than 6
4. Pickup date before 2021
5. Dropoff date > 31st Mar 2024
6. Duration of taxi ride > 5hrs
7. PU location ID not an allowed value
8. DO location ID not an allowed value



In [ ]:
# Test 1: 
df[df['tpep_dropoff_datetime']<df['tpep_pickup_datetime']]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
# Test 2:
df[df['passenger_count']<0]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
# Test 3:
df[df['passenger_count']>6]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
# Test 4: 
df[df['tpep_pickup_datetime']<pd.to_datetime('2021-01-01')]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
# Test 5: 
df[df['tpep_dropoff_datetime']>pd.to_datetime('2024-04-01')]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
late_dropoff = df[df['tpep_dropoff_datetime']>pd.to_datetime('2024-04-01')]
late_dropoff.describe().T

,count,mean,min,25%,50%,75%,max,std
tpep_pickup_datetime,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN
tpep_dropoff_datetime,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN
passenger_count,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Test 6:
time_diff = pd.Timedelta(hours=5)
df[df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'] > time_diff]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
long_rides = df[df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'] > time_diff]
long_rides['time_diff'] = long_rides['tpep_dropoff_datetime']-long_rides['tpep_pickup_datetime']
long_rides.describe().T

,count,mean,min,25%,50%,75%,max,std
tpep_pickup_datetime,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN
tpep_dropoff_datetime,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN
passenger_count,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time_diff,0,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [ ]:
# Test 7:
taxi_lookup = pd.read_csv('taxi_zone_lookup.csv')
taxi_zones = set(taxi_lookup['LocationID'])
df[~df['PULocationID'].isin(taxi_zones)]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID


In [ ]:
df[~df['DOLocationID'].isin(taxi_zones)]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
